In [5]:
import pandas as pd
df = pd.read_csv("DESeq2 Gene expression.tsv", sep="\t", index_col=0)
df.to_csv("random_genes.tsv", sep="\t")
#здесь приведена таблица, полученная после обработки в пакете DESeq2
#дальше производится ее дополнительная обработка и поиск дифф экспрессированных генов
df = df.loc[df["baseMean"] > 5]#убрали наименее экспрессированные гены
df = df.sort_values(by = 'padj', ascending = True)
df1 = df.loc[df["padj"] == 0]
df1

,baseMean,log2FoldChange,lfcSE,pvalue,padj
Gene,,,,,
RIMKLA,970.397019,3.335757,0.087771,0.0,0.0
NAA25,3830.781375,2.931441,0.058618,0.0,0.0
YWHAG,13209.569063,1.875891,0.044421,0.0,0.0
QSOX1,5851.677174,2.114584,0.047989,0.0,0.0
PSMC4,8547.850107,3.195610,0.044615,0.0,0.0
...,...,...,...,...,...
AHCYL1,6632.708020,1.693492,0.044534,0.0,0.0
DYNC1H1,15632.297567,2.187062,0.039142,0.0,0.0
ANKZF1,2298.112026,2.374386,0.061955,0.0,0.0


дифференциально экспрессированные гены по сортировке log2FoldChange при padj = 0 
(таблица выведена в предыдущем блоке)
и недиффиренциально экспрессированные гены по сортировке padj

In [10]:
diff_genes_goi = set(df1.sort_values(by = 'log2FoldChange', ascending = False).index[:5])#дифф гены
ref_genes = set(df.sort_values(by = 'padj', ascending = False).index[:11])#недифф гены
#Было взято больше 5 и 10 генов в связи с трудностями при выборе референсных генов, 
#функции многих из найдейнных 10 не были однозначно определены
print(diff_genes_goi)
print(ref_genes)

{'IGFL2-AS1', 'MAP1A', 'PFKFB4', 'NDRG1', 'HMOX1'}
{'FBXL13', 'KLHL32', 'RNF144A', 'AC008750.5', 'AP001542.3', 'WDR24', 'AL356299.3', 'AC007566.1', 'POLR2J2', 'BTN3A3', 'WDR89'}


решение искать дифф экспрессированные именно так тем, что padj -> 0 показывает статистически значимую дифф экспрессию.
Далее производилась сортировка по log2FoldChange для поиска наибольшей разницы среди отобранных генов.
Наименее дифференциально экспрессированные гены искались по обратной схеме: padj наибольший

посмотрим на степень экспрессированности выбранных генов (на столбец baseMean)
Его потом будем использовать для обоснования выбора референсных генов

In [9]:
df2 = df.loc[df.index.isin(['IGFL2-AS1', 'HMOX1', 'NDRG1', 'MAP1A', 'PFKFB4'])] #все целевые гены хорошо экспрессированы
df2

,baseMean,log2FoldChange,lfcSE,pvalue,padj
Gene,,,,,
ADM,1424.959990,5.402157,0.112207,0.0,0.0
IGFL2-AS1,1810.287000,6.059001,0.121143,0.0,0.0
NDRG1,67721.617721,5.958682,0.051851,0.0,0.0
HMOX1,16880.360560,8.747973,0.095660,0.0,0.0
PFKFB4,2118.742505,5.464637,0.097706,0.0,0.0
MAP1A,3009.589442,7.665786,0.146527,0.0,0.0


In [7]:
df3 = df.loc[df.index.isin(['FBXL13', 'KLHL32', 'RNF144A', 'AC008750.5', 'AP001542.3', 'WDR24', 'AL356299.3', 'AC007566.1', 'POLR2J2', 'BTN3A3', 'WDR89'])]
df3

,baseMean,log2FoldChange,lfcSE,pvalue,padj
Gene,,,,,
POLR2J2,6.495870,0.003502,0.610901,0.995562,0.997102
AP001542.3,6.337054,0.002517,0.578814,0.997077,0.998222
RNF144A,5.899481,-0.000667,0.594195,0.997206,0.998308
KLHL32,5.573886,-0.000354,0.605495,0.997769,0.998430
AC008750.5,11.749955,-0.000367,0.476327,0.997714,0.998430
AC007566.1,69.836456,-0.000138,0.230798,0.998329,0.998770
WDR89,146.556662,0.000501,0.164141,0.998583,0.998936
FBXL13,21.499365,0.001240,0.380244,0.998552,0.998936
BTN3A3,79.203653,0.000520,0.218108,0.999214,0.999435


Далее убедимся, что выбранные нами гены не самые высоко или низко экспрепрессируемые (так как такие брать не рекомендуется)
И это действительно не так: как видно топ 10 самых низко и высоко экспрессированных генов не совпадает с нашими

In [47]:
reads_most = set(df.sort_values(by = 'baseMean', ascending = False).index[:10])
reads_least = set(df.sort_values(by = 'baseMean', ascending = True).index[:10])
print(reads_most)
print(reads_least)

{'MT-ND4', 'MT-CO1', 'MT-CO3', 'MT-ND5', 'MT-CO2', 'FTL', 'MT-RNR2', 'MT-ATP6', 'GAPDH', 'EEF1A1'}
{'ARIH2P1', 'AC007611.1', 'AC009244.1', 'AC069224.2', 'KCNK15', 'VAV3', 'LTC4S', 'FCMR', 'PDXP', 'PCDHGA5'}


Дальнейшие объяснения и анализ найденных генов приведены в документе Word